In [1]:
#979984173 total
#!wc -l /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/data/userurl/gn/cm/jiangsu/nanjing/20131228/*/*

In [1]:
!head -n 1 /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/data/userurl/gn/cm/jiangsu/nanjing/20131228/39server/userurl_20131228_235801_001039.log

In [2]:
filePath='/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/data/userurl/gn/cm/jiangsu/nanjing/20131228/*/*'

In [3]:
lines = sc.textFile(filePath)
parts = lines.map(lambda l: l.split("\t")).filter(lambda line : len(line)==88)
parts.count()

979984173

In [4]:
def map_function(p):
    if p[38] == '-':
        return (p[4], [p[0], p[39],p[35],p[35],p[64],int(p[46])])
    else:
        return (p[4], [p[0], p[39],p[35]+p[38],p[35], p[64],int(p[46])])
user_flow=parts.map(lambda  p: map_function(p))
user_flow.count()

979984173

In [5]:
user_flow=user_flow.sortByKey()

In [6]:
user_flow.first()

('8613032261669',
 ['1388219305.310000',
  '-',
  'dns.weixin.qq.com/cgi-bin/micromsg-bin/newgetdns?uin=2046042280&clientversion=352321792&scene=0&net=0',
  'dns.weixin.qq.com',
  'http',
  4366])

In [7]:
user_flow.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw/user_flow")

# data explore

In [8]:
#count how many Packet have referer infomation
user_flow.filter(lambda u : u[1][1]!='-').count()

151437973

In [9]:
# the type of network 
from operator import add
user_flow.map(lambda u : (u[1][4],1)).reduceByKey(add).collect()

[('http', 799998850), ('wap2', 179985323)]

In [12]:
# rank the domain  
from operator import add
user_flow.map(lambda u : (u[1][3],1)).reduceByKey(add).sortBy(lambda x: x[1],ascending=False).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw/domain-rank")

In [15]:
%whos

Variable       Type                          Data/Info
------------------------------------------------------
add            builtin_function_or_method    <built-in function add>
filePath       str                           /media/jw/5dccc50e-7c13-4<...>ngsu/nanjing/20131228/*/*
flow_time      PipelinedRDD                  PythonRDD[45] at collect <...>-input-14-158c64e04297>:3
getKey         function                      <function getKey at 0x7f716196a620>
lines          RDD                           /media/jw/5dccc50e-7c13-4<...>ethodAccessorImpl.java:-2
map_function   function                      <function map_function at 0x7f718f57a730>
parts          PipelinedRDD                  PythonRDD[46] at RDD at PythonRDD.scala:48
user_flow      PipelinedRDD                  PythonRDD[47] at RDD at PythonRDD.scala:48


## load user flow

In [1]:
!head -n 1 /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw/user_flow/part-00000

('8613032261669', ['1388219305.310000', '-', 'dns.weixin.qq.com/cgi-bin/micromsg-bin/newgetdns?uin=2046042280&clientversion=352321792&scene=0&net=0', 'dns.weixin.qq.com', 'http', 4366])


In [1]:
user_flow = sc.textFile('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw/user_flow').map(lambda t : eval(t)).repartition(200)
type(user_flow.first())

tuple

In [2]:
user_flow.first()

('8613776545743',
 ['1388229722.730000',
  '-',
  'a288.phobos.apple.com/us/r30/Purple/v4/69/be/cc/69becc7c-1d0b-bb45-af10-8cde3fe42821/mzl.gafkqwxv.114x114-75.png',
  'a288.phobos.apple.com',
  'http',
  15269])

## domain2vec data prepare

In [1]:
user_flow = sc.textFile('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw/user_flow/part-00000').map(lambda t : eval(t))

In [2]:
domain_flow=user_flow.map(lambda u :(u[0],[u[1][0],u[1][3]]))
def add(a, b): 
    a.append(b)
    return a 
# def add(a, b): return a + str(b)
user_domain_flow=domain_flow.foldByKey([],add)
user_domain_flow.repartition(20)

MapPartitionsRDD[10] at repartition at NativeMethodAccessorImpl.java:-2

In [3]:
user_domain_flow.first()

('8613400001908',
 [['1388210437.226000', '183.218.12.66:12345'],
  ['1388212991.182000', 'wireless.mapbar.com'],
  ['1388213015.313000', '183.218.12.66:12345'],
  ['1388213124.050000', 'monitor.uu.qq.com'],
  ['1388209804.614000', 'monitor.uu.qq.com'],
  ['1388209913.325000', 'monitor.uu.qq.com'],
  ['1388212503.677000', '183.218.12.66:12345'],
  ['1388212653.253000', 'monitor.uu.qq.com'],
  ['1388212653.272000', 'monitor.uu.qq.com'],
  ['1388212707.992000', 'monitor.uu.qq.com'],
  ['1388213518.471000', 'monitor.uu.qq.com'],
  ['1388213515.780000', 'monitor.uu.qq.com'],
  ['1388213806.870000', 'monitor.uu.qq.com'],
  ['1388213817.761000', 'monitor.uu.qq.com'],
  ['1388209934.413000', 'monitor.uu.qq.com'],
  ['1388212333.634000', 'monitor.uu.qq.com'],
  ['1388212335.674000', 'monitor.uu.qq.com'],
  ['1388213346.131000', '183.218.12.66:12345'],
  ['1388210736.630000', '183.218.12.66:12345'],
  ['1388211419.818000', 'monitor.uu.qq.com'],
  ['1388211419.051000', 'monitor.uu.qq.com'],
  ['

In [6]:
user_domain_flow.repartition(200)
user_domain_flow.saveAsTextFile('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw/3')

## url2vec data prepare

In [ ]:
domain_flow=user_flow.map(lambda u :(u[0],[u[1][0],u[1][2]]))
def add(a, b): 
    a.append(b)
    return a 
# def add(a, b): return a + str(b)
user_domain_flow=domain_flow.foldByKey([],add)
user_domain_flow.repartition(100)